In [1]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

import datetime as dt # For defining dates

import time
# In Powershell Prompt : conda install -c conda-forge multitasking
# pip install -i https://pypi.anaconda.org/ranaroussi/simple yfinance

import yfinance as yf

# To show all your output File -> Preferences -> Settings Search for Notebook
# Notebook Output Text Line Limit and set to 100

# Used for file handling like deleting files
import os

# conda install -c conda-forge cufflinks-py
# conda install -c plotly plotly
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

from plotly.subplots import make_subplots

# New Imports
# Used to get data from a directory

from os import listdir
from os.path import isfile, join

import warnings
warnings.simplefilter("ignore")

Constants

In [2]:
PATH = "C:/Users/davez/Desktop/Investing with Data Science/"
S_DATE = "2017-02-01"
E_DATE = "2022-12-06"
S_DATE_DT = pd.to_datetime(S_DATE)
E_DATE_DT = pd.to_datetime(E_DATE)


Get Column Data from CSV

In [3]:
def get_column_from_csv(file, col_name):
    # Try to get the file and if it doesnt exist issue a warning
    try:
        df = pd.read_csv(file)
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        return df[col_name]

Get Stock Tickers

In [4]:
tickers = get_column_from_csv("C:/Users/davez/Desktop/Investing with Data Science/Wilshire-5000-Stocks-New.csv", "Ticker")
print(len(tickers))
print(tickers)

3481
0          A
1         AA
2        AAL
3       AAME
4        AAN
        ... 
3476    ZUMZ
3477     ZUO
3478    ZYNE
3479    ZYXI
3480    ZNGA
Name: Ticker, Length: 3481, dtype: object


Get Dataframe from CSV

In [5]:
# Reads a dataframe from the CSV file, changes index to date and returns it
def get_stock_df_from_csv(ticker):
    
    # Try to get the file and if it doesn't exist issue a warning
    try:
        df = pd.read_csv(PATH + ticker + '.csv', index_col=0)
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        return df

In [6]:
print(tickers)

0          A
1         AA
2        AAL
3       AAME
4        AAN
        ... 
3476    ZUMZ
3477     ZUO
3478    ZYNE
3479    ZYXI
3480    ZNGA
Name: Ticker, Length: 3481, dtype: object


In [7]:

files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
tickers
# tickers.remove('.ds_Store') MacOS Only
tickers.sort()
len(tickers)

3262

In [8]:
# Add Daily Returns
def add_daily_return_to_df(df):
    df['daily_return'] = (df['Close'] / df['Close'].shift(1)) - 1
    #df.to_csv(PATH + ticker + '.csv')
    return df

In [9]:
# Add Cumulative Returns
def add_cum_return_to_df(df):
    df['cum_return'] = (1 + df['daily_return']).cumprod()
    #df.to_csv(PATH + ticker + '.csv')
    return df

In [10]:
# Add Bollinger Bands
def add_bollinger_bands(df):
    df['middle_band'] = df['Close'].rolling(window=20).mean()
    df['upper_band'] = df['middle_band'] + 1.96 * df['Close'].rolling(window=20).std()
    df['lower_band'] = df['middle_band'] - 1.96 * df['Close'].rolling(window=20).std()
    return df
    # df.to_csv(PATH + ticker + '.csv')

In [11]:
# Add Ichimoku Data to Dataframe
def add_Ichimoku(df):
    # Conversion Line = (Highest value in period + Lowest value in period)/2 (9 sessions)
    hi_val = df['High'].rolling(window=9).max()
    low_val = df['Low'].rolling(window=9).min()
    df['Conversion'] = (hi_val + low_val) / 2
    
    # Base Line = (Highest value in period + Lowest value in period)/2 (26 sessions)
    hi_val2 = df['High'].rolling(window=26).max()
    low_val2 = df['Low'].rolling(window=26).min()
    df['Baseline'] = (hi_val2 + low_val2) / 2
    
    # Span A = (Conversion Value + Base Value)/2 (26 sessions)
    df['SpanA'] = ((df['Conversion'] + df['Baseline']) / 2)
    
    # Span B = (Conversion Value + Base Value)/2 (52 sessions)
    hi_val3 = df['High'].rolling(window=52).max()
    low_val3 = df['Low'].rolling(window=52).min()
    df['SpanB'] = ((hi_val3 + low_val3) / 2).shift(26)
    
    # Lagging Span = Price shifted back 26 periods
    df['Lagging'] = df['Close'].shift(-26)
    return df
    # df.to_csv(PATH + ticker + '.csv')

In [12]:
# Always a good idea to test using one stock first (rather than 3500)...
# so try on ticker A
try:
    print("Working on :", "A")
    new_df = get_stock_df_from_csv("A")
    new_df = add_daily_return_to_df(new_df)
    new_df = add_cum_return_to_df(new_df)
    new_df = add_bollinger_bands(new_df)
    new_df = add_Ichimoku(new_df)
    new_df.to_csv(PATH + 'A' + '.csv')
except Exception as ex:
    print(ex)

Working on : A


In [13]:
for x in tickers:
    try:
        print("Working on :", x)
        new_df = get_stock_df_from_csv(x)
        new_df = add_daily_return_to_df(new_df)
        new_df = add_cum_return_to_df(new_df)
        new_df = add_bollinger_bands(new_df)
        new_df = add_Ichimoku(new_df)
        new_df.to_csv(PATH + x + '.csv')
    except Exception as ex:
        print(ex)

Working on : A
Working on : AA
Working on : AAL
Working on : AAME
Working on : AAN
Working on : AAOI
Working on : AAON
Working on : AAP
Working on : AAPL
Working on : AAT
Working on : AAWW
Working on : AAXN
Working on : ABBV
Working on : ABC
Working on : ABCB
Working on : ABEO
Working on : ABG
Working on : ABIO
Working on : ABM
Working on : ABMD
Working on : ABR
Working on : ABT
Working on : ABTX
Working on : AC
Working on : ACA
Working on : ACAD
Working on : ACBI
Working on : ACC
Working on : ACCO
Working on : ACER
Working on : ACGL
Working on : ACHC
Working on : ACHV
Working on : ACIA
Working on : ACIW
Working on : ACLS
Working on : ACM
Working on : ACMR
Working on : ACN
Working on : ACNB
Working on : ACOR
Working on : ACRE
Working on : ACRS
Working on : ACRX
Working on : ACTG
Working on : ACU
Working on : ACY
Working on : ADBE
Working on : ADC
Working on : ADES
Working on : ADI
Working on : ADM
Working on : ADMA
Working on : ADMP
Working on : ADMS
Working on : ADNT
Working on : ADP


Working on : BXC
Working on : BXMT
Working on : BXP
Working on : BXS
Working on : BY
Working on : BYD
Working on : BYFC
Working on : BYND
Working on : BZH
Working on : C
Working on : CABO
Working on : CAC
Working on : CACC
Working on : CACI
Working on : CADE
Working on : CAG
Working on : CAH
Working on : CAI
Working on : CAKE
Working on : CAL
Working on : CALA
Working on : CALM
Working on : CALX
Working on : CAMP
Working on : CAPR
Working on : CAR
Working on : CARA
Working on : CARG
Working on : CARS
Working on : CASA
Working on : CASH
Working on : CASI
Working on : CASS
Working on : CASY
Working on : CAT
Working on : CATB
Working on : CATC
Working on : CATM
Working on : CATO
Working on : CATS
Working on : CATY
Working on : CB
Working on : CBAN
Working on : CBAY
Working on : CBB
Working on : CBFV
Working on : CBIO
Working on : CBLI
Working on : CBMG
Working on : CBOE
Working on : CBRE
Working on : CBRL
Working on : CBS
Working on : CBSH
Working on : CBT
Working on : CBTX
Working on : C

Working on : EIG
Working on : EIGI
Working on : EIGR
Working on : EIX
Working on : EKSO
Working on : EL
Working on : ELAN
Working on : ELF
Working on : ELGX
Working on : ELMD
Working on : ELOX
Working on : ELS
Working on : ELSE
Working on : ELVT
Working on : ELY
Working on : EMAN
Working on : EMCF
Working on : EME
Working on : EMKR
Working on : EML
Working on : EMMS
Working on : EMN
Working on : EMR
Working on : ENDP
Working on : ENG
Working on : ENOB
Working on : ENPH
Working on : ENR
Working on : ENS
Working on : ENSG
Working on : ENSV
Working on : ENT
Working on : ENTA
Working on : ENTG
Working on : ENV
Working on : ENVA
Working on : ENZ
Working on : EOG
Working on : EOLS
Working on : EPAM
Working on : EPAY
Working on : EPC
Working on : EPM
Working on : EPR
Working on : EPRT
Working on : EPZM
Working on : EQBK
Working on : EQC
Working on : EQH
Working on : EQIX
Working on : EQR
Working on : EQT
Working on : ERI
Working on : ERIE
Working on : ERII
Working on : ES
Working on : ESBK
Wo

Working on : HSON
Working on : HST
Working on : HSTM
Working on : HSY
Working on : HT
Working on : HTA
Working on : HTBI
Working on : HTBK
Working on : HTBX
Working on : HTGM
Working on : HTH
Working on : HTLD
Working on : HTLF
Working on : HUBB
Working on : HUBG
Working on : HUBS
Working on : HUM
Working on : HUN
Working on : HURC
Working on : HURN
Working on : HUSA
Working on : HVT
Working on : HWBK
Working on : HWC
Working on : HWCC
Working on : HWKN
Working on : HXL
Working on : HY
Working on : HZN
Working on : HZNP
Working on : HZO
Working on : IAC
Working on : IART
Working on : IBCP
Working on : IBIO
Working on : IBKC
Working on : IBKR
Working on : IBM
Working on : IBOC
Working on : IBP
Working on : IBTX
Working on : ICAD
Working on : ICBK
Working on : ICCC
Working on : ICD
Working on : ICE
Working on : ICFI
Working on : ICHR
Working on : ICON
Working on : ICPT
Working on : ICUI
Working on : IDA
Working on : IDCC
Working on : IDN
Working on : IDRA
Working on : IDT
Working on : ID

Working on : MLP
Working on : MLR
Working on : MLSS
Working on : MLVF
Working on : MMAC
Working on : MMC
Working on : MMI
Working on : MMM
Working on : MMS
Working on : MMSI
Working on : MN
Working on : MNK
Working on : MNKD
Working on : MNOV
Working on : MNR
Working on : MNRL
Working on : MNRO
Working on : MNST
Working on : MNTA
Working on : MNTX
Working on : MO
Working on : MOBL
Working on : MOD
Working on : MODN
Working on : MOFG
Working on : MOH
Working on : MORN
Working on : MOS
Working on : MOSY
Working on : MOTS
Working on : MOV
Working on : MPAA
Working on : MPB
Working on : MPC
Working on : MPW
Working on : MPWR
Working on : MPX
Working on : MRAM
Working on : MRC
Working on : MRCY
Working on : MRIN
Working on : MRK
Working on : MRKR
Working on : MRLN
Working on : MRNS
Working on : MRO
Working on : MRSN
Working on : MRTN
Working on : MRTX
Working on : MRVL
Working on : MS
Working on : MSA
Working on : MSBF
Working on : MSBI
Working on : MSCI
Working on : MSEX
Working on : MSFT


Working on : PTC
Working on : PTCT
Working on : PTE
Working on : PTEN
Working on : PTGX
Working on : PTI
Working on : PTLA
Working on : PTN
Working on : PTSI
Working on : PTVCA
Working on : PTVCB
Working on : PUB
Working on : PULM
Working on : PUMP
Working on : PVAC
Working on : PVBC
Working on : PVH
Working on : PW
Working on : PWOD
Working on : PWR
Working on : PXD
Working on : PXLW
Working on : PYPL
Working on : PZG
Working on : PZN
Working on : PZZA
Working on : QADA
Working on : QADB
Working on : QCOM
Working on : QCRH
Working on : QDEL
Working on : QEP
Working on : QLYS
Working on : QNST
Working on : QRHC
Working on : QRTEA
Working on : QRVO
Working on : QTRX
Working on : QTS
Working on : QTWO
Working on : QUAD
Working on : QUIK
Working on : QUMU
Working on : QUOT
Working on : R
Working on : RAD
Working on : RAIL
Working on : RAMP
Working on : RARE
Working on : RAVE
Working on : RAVN
Working on : RBB
Working on : RBBN
Working on : RBC
Working on : RBCAA
Working on : RBCN
Working 

Working on : TEN
Working on : TENB
Working on : TENX
Working on : TER
Working on : TERP
Working on : TESS
Working on : TEUM
Working on : TEX
Working on : TFSL
Working on : TFX
Working on : TG
Working on : TGEN
Working on : TGI
Working on : TGLS
Working on : TGNA
Working on : TGT
Working on : TGTX
Working on : THC
Working on : THFF
Working on : THG
Working on : THO
Working on : THR
Working on : THRM
Working on : THS
Working on : TIF
Working on : TILE
Working on : TIPT
Working on : TISI
Working on : TITN
Working on : TJX
Working on : TKR
Working on : TLF
Working on : TLGT
Working on : TLRD
Working on : TLYS
Working on : TMDX
Working on : TMHC
Working on : TMO
Working on : TMP
Working on : TMST
Working on : TMUS
Working on : TNAV
Working on : TNC
Working on : TNDM
Working on : TNET
Working on : TNXP
Working on : TOL
Working on : TORC
Working on : TOWN
Working on : TPB
Working on : TPC
Working on : TPH
Working on : TPHS
Working on : TPIC
Working on : TPR
Working on : TPRE
Working on : TPTX

Plot Bollinger Bands

In [14]:
def plot_with_boll_bands(df, ticker):
    fig = go.Figure()
    
    candle = go.Candlestick(x=df.index, open=df['Open'],
    high=df['High'], low=df['Low'],
    close=df['Close'], name='Candlestick')
    
    upper_line = go.Scatter(x=df.index, y=df['upper_band'], 
    line=dict(color='rgba(250, 0, 0, 0.75)', 
    width=1), name="Upper Band")
    
    mid_line = go.Scatter(x=df.index, y=df['middle_band'],
    line=dict(color='rgba(0, 0, 250, 0.75)',
    width=0.7), name='Middle Band') 
    
    lower_line = go.Scatter(x=df.index, y=df['lower_band'],
    line=dict(color='rgba(0, 250, 0, 0.75)',
    width=1), name='Lower Band') 
    
    fig.add_trace(candle)
    fig.add_trace(upper_line)
    fig.add_trace(mid_line)
    fig.add_trace(lower_line)
    
    fig.update_xaxes(title='Date', rangeslider_visible=True)
    fig.update_yaxes(title='Price')
    
    fig.update_layout(title=ticker + "Bollinger Bands",
    height=900, width=1500, showlegend=True)
    fig.show()

In [15]:
test_df = get_stock_df_from_csv('AMD')
test_df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,daily_return,cum_return,middle_band,upper_band,lower_band,Conversion,Baseline,SpanA,SpanB,Lagging
Date,,,,,,,,,,,,,,,,,
2017-02-01,10.90,12.14,10.81,12.06,165939300,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.91
2017-02-02,11.98,12.66,11.95,12.28,116324500,0,0,0.018242,1.018242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.28
2017-02-03,12.37,12.50,12.04,12.24,60036700,0,0,-0.003257,1.014925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.10
2017-02-06,12.46,13.70,12.38,13.63,140474100,0,0,0.113562,1.130182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.98
2017-02-07,14.05,14.27,13.06,13.29,158683800,0,0,-0.024945,1.101990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.65


In [16]:
#test_df = get_stock_df_from_csv('AMD')
plot_with_boll_bands(test_df, 'AAOI')